# Проект "Музыка больших городов" на основе данных Яндекс.Музыка


 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.



## Обзор данных





In [1]:
# импорт библиотеки pandas
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/Users/l.stefanenko/Desktop/Yandex/первые проекты/1  музыка скачано/yandex_music_project.csv')

In [3]:
# получение первых 10 строк таблицы df
df.head (10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


### Вывод обзора датасета:

<div class="alert alert-info"> 
 
В таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Названия напианы не в snake_case регистре 



Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


<div class="alert alert-info"> 

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных


Названия столбцов:

In [5]:
df.columns 

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

#### Переименование столбцов

In [6]:
df = df.rename (columns = {'  userID':'user_id',
                        'Track':'track',
                        '  City  ':'city',
                        'Day':'day'}) 

#### Проверка результатов - перечень названий столбцов

In [7]:
df.columns 

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений

In [8]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге.
* заполним эти пропуски явными обозначениями,
* оценим, насколько они повредят расчётам. 

### Замена пропущенных значений в столбцах `track`, `artist` и `genre` на строку `'unknown'`.

In [9]:
columns_to_replace = ['track', 'artist', 'genre']
for index in columns_to_replace:
    df [index] = df [index].fillna ('unknown')

In [10]:
# подсчёт пропусков после замены
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты

### Подсчёт явных дубликатов

In [11]:
df.duplicated().sum()

3826

#### Удаление явных дубликатов (с удалением старых индексов и формированием новых)

In [12]:
df = df.drop_duplicates().reset_index(drop = True)

In [13]:
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

Теперь избавимсяя от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному.

In [14]:
df = df.sort_values(by = 'genre', ascending = True)
df['genre'].unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Найдены неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу, напишем функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.

Функция должна исправить колонку `genre` в таблице `df`: заменить каждое значение из списка `wrong_genres` на значение из `correct_genre`.

### Функция для замены неявных дубликатов

In [15]:
def replace_wrong_genres(wrong_genres,correct_genre):
    for wrong_genre in wrong_genres:
        df ['genre'] = df ['genre'].replace(wrong_genre,correct_genre)

### Устранение неявных дубликатов

In [16]:
wrong_genres = ['hip','hop','hip-hop']
correct_genre = 'hiphop'
replace_wrong_genres (wrong_genres,correct_genre)

wrong_genres = ['электроника']
correct_genre = 'electronic'
replace_wrong_genres (wrong_genres,correct_genre)

In [17]:
# Проверка на наличие неявных дубликатов
df['genre'].unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

<div class="alert alert-info">
Вывод

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Пропущенные значения заменены на `'unknown'`.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверьте это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделите пользователей Москвы и Санкт-Петербурга
* Сравните, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


#### Подсчёт количетва прослушиваний в каждом городе

In [18]:
df.groupby('city')['track'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: track, dtype: int64

<div class="alert alert-info">
Вывод
    
В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируйте данные по дню недели и подсчитайте прослушивания в понедельник, среду и пятницу. Учтите, что в данных есть информация только о прослушиваниях только за эти дни.


In [19]:
df.groupby('day')['track'].count()# Подсчёт прослушиваний в каждый из трёх дней

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [20]:

def number_tracks(day,city): # Объявляется функция с двумя параметрами: day, city.
    track_list = df[(df['day'] == day) & (df['city'] == city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [21]:
print ('Количество прослушиваний в Москве по понедельникам:', number_tracks('Monday','Moscow'))# количество прослушиваний в Москве по понедельникам

Количество прослушиваний в Москве по понедельникам: 15740


In [22]:
print ('Количество прослушиваний в Санкт-Петербурге по понедельникам:',number_tracks('Monday','Saint-Petersburg'))# количество прослушиваний в Санкт-Петербурге по понедельникам

Количество прослушиваний в Санкт-Петербурге по понедельникам: 5614


In [23]:
print ('Количество прослушиваний в Москве по средам:', number_tracks('Wednesday','Moscow'))# количество прослушиваний в Москве по средам

Количество прослушиваний в Москве по средам: 11056


In [24]:
print ('Количество прослушиваний в Санкт-Петербурге по средам:', number_tracks('Wednesday','Saint-Petersburg'))# количество прослушиваний в Санкт-Петербурге по средам

Количество прослушиваний в Санкт-Петербурге по средам: 7003


In [25]:
print ('Количество прослушиваний в Москве по пятницам:', number_tracks('Friday','Moscow'))# количество прослушиваний в Москве по пятницам

Количество прослушиваний в Москве по пятницам: 15945


In [26]:
print ('Количество прослушиваний в Санкт-Петербурге по пятницам:', number_tracks('Friday','Saint-Petersburg'))# количество прослушиваний в Санкт-Петербурге по пятницам

Количество прослушиваний в Санкт-Петербурге по пятницам: 5895


** Создание таблицы с выводом 
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [27]:
columns = ['city', 'monday', 'wednesday', 'friday']
data = [['Moscow','15740','15945','11056'],['Saint-Petersburg','7003','5614','5895']]
print(pd.DataFrame(data = data , columns = columns)) # Таблица с результатами

               city monday wednesday friday
0            Moscow  15740     15945  11056
1  Saint-Petersburg   7003      5614   5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраните таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [28]:
moscow_general = df[df['city'].isin (['Moscow'])]
print (moscow_general)

# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'


        user_id              track                 artist      genre    city  \
31916  C40BC5DF   Shot in the Dark             Four80East       acid  Moscow   
57580  23C64044  Don't Stop Me Now  Acoustic Heartstrings   acoustic  Moscow   
20264  B02ED3AB              Ой да                unknown   acoustic  Moscow   
37784  77D3A7D9              Ой да                unknown   acoustic  Moscow   
26378  4D96EB78        Eventuality          Jack Trammell     action  Moscow   
...         ...                ...                    ...        ...     ...   
22265  9E13CAB3           Caravana       Orchestra Baobab      world  Moscow   
7581   E9537159      Back To Black          Simone Mazzer      world  Moscow   
41879  613C79DF          Rani Mrid                  Nemri      world  Moscow   
27939  D7FB50DA    Drumming Circle       Professor Trance  worldbeat  Moscow   
8448   A439123F            Flip It                unknown        ïîï  Moscow   

           time        day  
31916  09:

In [29]:
spb_general = df[df['city'].isin (['Saint-Petersburg'])]
print(spb_general)# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'


        user_id                       track                artist       genre  \
3636   F10919ED          Ready For The Fire      Valley Of Wolves    acoustic   
14611  1C7D9E90  Prelude – Ne Me Qui Te Pas               unknown    acoustic   
9393   B24158ED                  Cellophane   Sara Jackson-Holman       adult   
13408   BF3BEA4                  Sweet Love           Anita Baker       adult   
34021  4DCD818F            Blue Skies Again  Jessica Lea Mayfield       adult   
...         ...                         ...                   ...         ...   
33671  CB06B200                   Sapashini             Stonebwoy       world   
7465   CCC16A51   Toccata Aal Lebnene Remix         Michel Fadel        world   
7576   5E9261F4                  Wah Yantee                Simrit       world   
39063  AA1730E8                         Anu     Kailash Kokopelli   worldbeat   
6465     1D4661                 Раз два три                 Какао  electronic   

                   city    

Создайте функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [30]:
def genre_weekday(table, day, time1, time2):
    genre_df = table[(table['day'] == day) & (df['time'] > time1) & (df['time'] < time2)]
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending = False).head(10)
    return  genre_df_sorted

Cравните результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [39]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [32]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [33]:
genre_weekday(moscow_general, 'Friday','17:00', '23:00')# вызов функции для вечера пятницы в Москве

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [34]:
genre_weekday(spb_general, 'Friday','17:00', '23:00')# вызов функции для вечера пятницы в Петербурге

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируйте таблицу `moscow_general` по жанру и посчитайте прослушивания треков каждого жанра методом `count()`. Затем отсортируйте результат в порядке убывания и сохраните его в таблице `moscow_genres`.

In [35]:
moscow_genres = moscow_general.groupby('genre')['genre'].count()
moscow_genres = moscow_genres.sort_values(ascending = False)

In [36]:
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Группировка таблицы `spb_general` по жанру. Подсчет количства прослушивания треков каждого жанра. Отсортировка результата в порядке убывания в `spb_genres`:

In [37]:
spb_genres = spb_general.groupby('genre')['genre'].count()
spb_genres = spb_genres.sort_values(ascending = False)

In [38]:
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

<div class="alert alert-info"> Гипотезы:

 > 1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
 >2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
 >3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

<b>Результаты проверки гипотез:</b>

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

<b>Первая гипотеза полностью подтвердилась.</b>

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
> * в Москве преимущественно слушают музыку жанра world
> * в Санкт - Петербурге больше любителей джаза и классичекой музыки

<b>Вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.</b>

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

<b>Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны. </b>
